# Code

## Needed modules

In [26]:
# Needed modules
import os
import subprocess
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

## Load data

In [27]:
# Load al the data
df_main = pd.read_csv("./data/consensus+LmSIDER2A+B/AL_DATA.csv", sep=",", header=0)
print(df_main.shape)
print(df_main.dtypes)
df_main.head()

(2092, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...
1,LinJ.01,24093,25080,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...
2,LinJ.01,35371,36297,plus,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...
3,LinJ.01,39790,40595,plus,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...
4,LinJ.01,54983,55909,plus,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...


In [28]:
# Create a column with the length of the sseq
df_main["length"] = df_main["sseq"].apply(lambda x: len(x))

In [29]:
# Check df
df_main.head()

,sseqid,sstart,send,sstrand,sseq,length
0,LinJ.01,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...,1000
1,LinJ.01,24093,25080,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...,988
2,LinJ.01,35371,36297,plus,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927
3,LinJ.01,39790,40595,plus,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...,806
4,LinJ.01,54983,55909,plus,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927


In [30]:
# check dtypes
df_main.dtypes

sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
length      int64
dtype: object

In [31]:
# describe length column
df_main["length"].describe()

count     2092.000000
mean       827.937859
std        467.971245
min        101.000000
25%        649.000000
50%        770.000000
75%        921.000000
max      10546.000000
Name: length, dtype: float64

In [32]:
# Le'ts check the length of the elements above the third quartile
long_seqs = df_main[df_main["length"] > int(df_main["length"].describe()["75%"])].copy()
long_seqs

,sseqid,sstart,send,sstrand,sseq,length
0,LinJ.01,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...,1000
1,LinJ.01,24093,25080,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...,988
2,LinJ.01,35371,36297,plus,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927
4,LinJ.01,54983,55909,plus,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927
10,LinJ.01,145322,146653,plus,ATCCCCACGGGGTGGTGAGGCGAGAAGTCAGGGGTCGGGCACGCGC...,1332
...,...,...,...,...,...,...
2074,LinJ.36,2498408,2502567,plus,ACAACAAAACTGACGCTATTGAAAGCGGCTCTCGAGAAGCTTTCCT...,4160
2075,LinJ.36,2504116,2505057,plus,TGTGTCCATTCTCTGCCACACAACATGAGCTAAGCTCTACTCTGCC...,942
2076,LinJ.36,2533606,2534550,plus,CAACGTGTCCATTCTCTGCCACACAACATGAGCTAAGCTCTACTCT...,945
2081,LinJ.36,2600817,2601884,plus,GCGCATGCCGAGCACCGCTGGCATGTGGTGTGCCGCATCCGACCGA...,1068


In [33]:
long_seqs_values = pd.DataFrame(long_seqs["length"].value_counts(sort=False))

In [34]:
long_seqs_values.sort_index(inplace=True, ascending=False)
long_seqs_values

,count
length,
10546,1
5635,1
4239,1
4160,1
4096,1
...,...
927,5
926,2
924,1


Now I have two values:
- `df_main` ==> with the 2092 elements
- `long_seqs`==> subset of  `df_main` with the elements with a length >Q3

## Main code

### Prepare functions

In [35]:
# Prepare functions
# Needed functions
def fasta_creator(csv_input, output_path):
    matrix = []
    for index, row in csv_input.iterrows():
        rec = SeqRecord(Seq(row["sseq"]), 
                        id = f"Seq_{index}_{row['sseqid']}",
                        description = "Leishmania infantum"
                        )
        matrix.append(rec)
    SeqIO.write(matrix, output_path, "fasta")

def blastn_dic(path_input, path_output):
    # "parse_seqids" is used to keep the sequence ID in the output.
    cmd = f"makeblastdb -in {path_input} -dbtype nucl -parse_seqids -out {path_output}"
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def blastn_blaster(query_path, dict_path):
    cmd = "blastn -word_size 11" \
        + " -query " + query_path \
        + " -db " + dict_path \
        + " -outfmt '10 qseqid sseqid sstrand pident qstart qend sstart send evalue bitscore length qlen qcovs slen mismatch gapopen gaps'"
    data = subprocess.run(cmd, shell=True, capture_output=True, text=True, universal_newlines=True, executable='/usr/bin/bash')  # Important the E value
    data = data.stdout
    data = pd.DataFrame([x.split(",") for x in data.split("\n") if x])
    if not data.empty:  # If the dataframe is not empty
        data.columns = ["qseqid", "sseqid", "sstrand", "pident", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "length", "qlen", "qcovs", "slen", "mismatch", "gapopen", "gaps"]
        data[['pident',  'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'qlen', 'qcovs', 'slen', 'mismatch', 'gapopen', 'gaps']] = data[['pident',  'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'qlen', 'qcovs', 'slen', 'mismatch', 'gapopen', 'gaps']].apply(pd.to_numeric)
    else:  # If the dataframe is empty
        data = pd.DataFrame(columns=["qseqid", "sseqid", "sstrand", "pident", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "length", "qlen", "qcovs", "slen", "mismatch", "gapopen", "gaps"])  # Create an empty dataframe
    return data  

### Prepare paths

In [37]:
# Prepare paths
path_genome = "../0.Data/genome/L_infantum/TriTrypDB-67_LinfantumJPCM5_Genome.fasta"

### Prepare blastn dicts

In [ ]:
blastn_dic(path_input=path_genome, path_output=path_genome)

### Test normal sequence

In [41]:
df_main.iloc[1,:]

sseqid                                               LinJ.01
sstart                                                 24093
send                                                   25080
sstrand                                                 plus
sseq       GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...
length                                                   988
Name: 1, dtype: object

In [49]:
test_seq_1 = {}
test_seq_1[f"Seq_{df_main.loc[1,'sseqid']}_{df_main.loc[1,'sstrand']}_{df_main.loc[1,'sstart']}-{df_main.loc[1,'send']}"] = df_main.loc[1, "sseq"]
test_seq_1

{'Seq_LinJ.01_plus_24093-25080': 'GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATCCCAGGGTCCAGCGCCCCCCCCCTCCACCCCCGCTCTCTCTGTGTACGGAAGCCCGGCAGCCCCCTCACCCTCTATCCCTGCCAACGCCGAACCACTTCTGGTGCTGACAGGGCTGGGCCCTTACGGCGTTGGGCGAGGTCGGCGCGACGTAGCGCCACGGATGCCGGCGGCCATGTCGTGCATGGCGCTGCGTCCGAGCCACCCGCGACAGTGAGCACAGGCTTGCACGGTCCATGCGATGGGCGGAGCGTGTCAGCGCGGCTCGAGCGTGTCGCAGCCGGCCCCTCACACTGGCCCACGGCGAGGGGTGCGGGGGCCTGAGTGTCACCGCGACGGGGAGACGCACCCAGGGAGGAGGTGGGGGGAGTGGGGACCGGCATGATGGAGGGGCGGCTGTGTGGCGATGTGTGTGTATGTGGGTGTCGTCGCGTTTGAGGCAGGAGCCGTGCTGCGACGACCGAGCCGGCGCACTGCTGCAGCGCGCGTGTGTCTTGCGCTGCTTCGCACCAGGCGATGAGAGTGGGGTGGGGTGCCTGCAGCACTCGGCGGCGGGGGGGTGCAGAGCGGCCTCCACCTCGCAGTGTGCGGCAGCGAAAACGGACACGCGGAAGAGCAAACACCAAGCCCGCACCCCTCTCCTCAGCCTCAGGAACGGGCTCCTAAAACGGCTCGATGCTATCGAGCTCCGCCTGCGCAAAGGGCGCGGCACCATGCGCCACACCGCTCTCAGACAGGAGTCGCAAGATGGAGTTCACAGCAACGCGATACGACGGCGCGGCTACCTCTGTCAGCATATTGGTGCGGCGGTACGCCTCCATCGCGACGCGGCGCATCTCACTGTGCGAAGAGAGGCGCCGCTCCTCTGCGTCCTCAGGAGCACGCCTGCGGATACGCGCCACCCGCACCGCCTCATCGACGC

#### Test GFF and coor

In [83]:
def fasta_creator2(sequence, fasta_output_path):
    rec = SeqRecord(Seq(sequence),
                    id="test"
                    )
    SeqIO.write(rec, fasta_output_path, "fasta")

def gff_creator(data_input, outh_path):
    # take only needed values
    data = data_input[["qseqid", "qstart", "qend"]].copy()
    # Prepare GFF type df
    gff_df = pd.DataFrame({
        'seqname': "test",
        'source': "CBM-302",
        'feature': "SIDER_test",  # Ensure this is the correct feature name
        'start': data.loc[:,"qstart"],
        'end': data.loc[:,"qend"],
        'score': ".",  # Placeholder if no score data
        'strand': ".",
        'frame': ".", # Placeholder if no frame data
        'attribute': "."
          })
    # Save to file
    gff_df.to_csv(outh_path, sep="\t", header=False, index=False)

def df_blasting_gff(dict, genome_path, out_path):  # version with csv
    name_id = list(dict.keys())[0]
    seq = list(dict.values())[0]
    query = f"<(echo -e '>{name_id}\n{seq}')"  # Create a query
    data = blastn_blaster(query_path=query, dict_path=genome_path)
    # data = data[data["evalue"] <= 1.0e-30]
    data = data[data["evalue"] <= 1.0e-09]
    data = data[(data["length"] >= 100) & (data["length"] <= 1500)]
    gff_creator(data, out_path)
    return data

def gff_viewer(main_df, slice_num, path_folder, path_genome):
    main_dict = {}
    main_dict[f"Seq_{main_df.loc[slice_num,'sseqid']}_{main_df.loc[slice_num,'sstrand']}_{main_df.loc[slice_num,'sstart']}-{main_df.loc[slice_num,'send']}"] = main_df.loc[slice_num, "sseq"]
    print(f"Analyzing sequence {slice_num} with length {len(main_df.loc[slice_num, 'sseq'])}")
    os.makedirs(path_folder, exist_ok=True)
    path_out_fasta = os.path.join(path_folder, "test.fasta")
    path_out_gff = os.path.join(path_folder, "test.gff")
    fasta_creator2(sequence=list(main_dict.values())[0], 
                   fasta_output_path=path_out_fasta)
    blastn_df = df_blasting_gff(dict=main_dict, 
                                genome_path=path_genome, 
                                out_path=path_out_gff)
    gff_df = pd.read_csv(path_out_gff, sep="\t", header=None)
    gff_df["length"] = gff_df[4] - gff_df[3] + 1
    gff_df.sort_values(by="length", ascending=False, inplace=True)
    print(f"""
    The longest sequences are:
        {gff_df.head(10)}
          """)
    return blastn_df


In [84]:
# create xml file
df_988 = gff_viewer(main_df=df_main, slice_num=1, path_folder="./test_1", path_genome=path_genome)

Analyzing sequence 1 with length 988

    The longest sequences are:
                0        1           2   3    4  5  6  7  8  length
0    test  CBM-302  SIDER_test   1  988  .  .  .  .     988
1    test  CBM-302  SIDER_test  23  666  .  .  .  .     644
2    test  CBM-302  SIDER_test   7  624  .  .  .  .     618
3    test  CBM-302  SIDER_test  30  477  .  .  .  .     448
43   test  CBM-302  SIDER_test  23  387  .  .  .  .     365
40   test  CBM-302  SIDER_test  23  387  .  .  .  .     365
42   test  CBM-302  SIDER_test  38  387  .  .  .  .     350
41   test  CBM-302  SIDER_test  38  387  .  .  .  .     350
214  test  CBM-302  SIDER_test  72  386  .  .  .  .     315
208  test  CBM-302  SIDER_test  76  386  .  .  .  .     311
          


In [132]:
df_main.head()

,sseqid,sstart,send,sstrand,sseq,length
0,LinJ.01,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...,1000
1,LinJ.01,24093,25080,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...,988
2,LinJ.01,35371,36297,plus,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927
3,LinJ.01,39790,40595,plus,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...,806
4,LinJ.01,54983,55909,plus,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927


In [135]:
gff_viewer(main_df=df_main, slice_num=5, path_folder="./test", path_genome=path_genome)

Analyzing sequence 5 with length 711

    The longest sequences are:
                0        1           2  3    4  5  6  7  8  length
0    test  CBM-302  SIDER_test  1  711  .  .  .  .     711
3    test  CBM-302  SIDER_test  1  438  .  .  .  .     438
1    test  CBM-302  SIDER_test  1  438  .  .  .  .     438
2    test  CBM-302  SIDER_test  1  438  .  .  .  .     438
254  test  CBM-302  SIDER_test  1  348  .  .  .  .     348
256  test  CBM-302  SIDER_test  1  348  .  .  .  .     348
253  test  CBM-302  SIDER_test  1  348  .  .  .  .     348
206  test  CBM-302  SIDER_test  1  347  .  .  .  .     347
205  test  CBM-302  SIDER_test  1  347  .  .  .  .     347
255  test  CBM-302  SIDER_test  4  348  .  .  .  .     345
          


### Test sequence big

In [137]:
long_seqs_values

,count
length,
10546,1
5635,1
4239,1
4160,1
4096,1
...,...
927,5
926,2
924,1


In [136]:
long_seqs 

,sseqid,sstart,send,sstrand,sseq,length
0,LinJ.01,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...,1000
1,LinJ.01,24093,25080,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...,988
2,LinJ.01,35371,36297,plus,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927
4,LinJ.01,54983,55909,plus,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927
10,LinJ.01,145322,146653,plus,ATCCCCACGGGGTGGTGAGGCGAGAAGTCAGGGGTCGGGCACGCGC...,1332
...,...,...,...,...,...,...
2074,LinJ.36,2498408,2502567,plus,ACAACAAAACTGACGCTATTGAAAGCGGCTCTCGAGAAGCTTTCCT...,4160
2075,LinJ.36,2504116,2505057,plus,TGTGTCCATTCTCTGCCACACAACATGAGCTAAGCTCTACTCTGCC...,942
2076,LinJ.36,2533606,2534550,plus,CAACGTGTCCATTCTCTGCCACACAACATGAGCTAAGCTCTACTCT...,945
2081,LinJ.36,2600817,2601884,plus,GCGCATGCCGAGCACCGCTGGCATGTGGTGTGCCGCATCCGACCGA...,1068


In [141]:
long_seqs[long_seqs["length"] > 4000]

,sseqid,sstart,send,sstrand,sseq,length
610,LinJ.19,608398,612636,plus,GCTACTGCTATGTCGGCGCTTAGGCCGTGGGTGGGAGCTGCATTGG...,4239
1708,LinJ.34,808220,813854,plus,ACGACGCAGCACAGCGGACGAGGAGAGAGGAGGCGGTAGGCAGGAG...,5635
1709,LinJ.34,813947,824492,plus,GGGACGGATAGACGGGGAGAGACATGCGGGGGAGGCAGTGGTGACG...,10546
1749,LinJ.34,1242905,1247000,plus,GACGCGCGCCTCCCCTCGACCCCCGCTGTCGCACACGGCATGCGCG...,4096
2074,LinJ.36,2498408,2502567,plus,ACAACAAAACTGACGCTATTGAAAGCGGCTCTCGAGAAGCTTTCCT...,4160


In [43]:
df_4160 = gff_viewer(main_df=long_seqs, slice_num=2074, path_folder="./test_2074_v2", path_genome=path_genome)

Analyzing sequence 2074 with length 4160

    The longest sequences are:
                0        1           2     3     4  5  6  7  8  length
0    test  CBM-302  SIDER_test     1  4160  .  .  .  .    4160
1    test  CBM-302  SIDER_test  2269  3546  .  .  .  .    1278
2    test  CBM-302  SIDER_test  2680  3878  .  .  .  .    1199
367  test  CBM-302  SIDER_test  2693  3828  .  .  .  .    1136
208  test  CBM-302  SIDER_test  2694  3822  .  .  .  .    1129
46   test  CBM-302  SIDER_test  2696  3823  .  .  .  .    1128
47   test  CBM-302  SIDER_test  2696  3823  .  .  .  .    1128
59   test  CBM-302  SIDER_test   734  1836  .  .  .  .    1103
60   test  CBM-302  SIDER_test   734  1836  .  .  .  .    1103
467  test  CBM-302  SIDER_test  2792  3820  .  .  .  .    1029
          


In [77]:
df_10546 = gff_viewer(main_df=long_seqs, slice_num=1709, path_folder="./test_1709", path_genome=path_genome)

Analyzing sequence 1709 with length 10546

    The longest sequences are:
               0        1           2     3      4  5  6  7  8  length
14  test  CBM-302  SIDER_test  6508   7755  .  .  .  .    1248
15  test  CBM-302  SIDER_test  9275  10521  .  .  .  .    1247
13  test  CBM-302  SIDER_test   974   2220  .  .  .  .    1247
6   test  CBM-302  SIDER_test  3741   4987  .  .  .  .    1247
5   test  CBM-302  SIDER_test  6508   7682  .  .  .  .    1175
17  test  CBM-302  SIDER_test  6508   7682  .  .  .  .    1175
4   test  CBM-302  SIDER_test  6508   7682  .  .  .  .    1175
3   test  CBM-302  SIDER_test  6508   7682  .  .  .  .    1175
12  test  CBM-302  SIDER_test  9275  10448  .  .  .  .    1174
16  test  CBM-302  SIDER_test  3741   4914  .  .  .  .    1174
          


In [10]:
# Le't try to merge that data from the sequence 10546
test_path = "./test_1709"
gff_df = pd.read_csv(os.path.join(test_path, "test.gff"), sep="\t", header=None)

In [17]:
# save to bed files
def gff_to_bed_intersect(input_df, gff_out_path):
    gff_df_bed = gff_df[[0, 3, 4]].copy()
    gff_df_bed.columns = ["seqname", "start", "end"]
    gff_df_bed.sort_values(by="start", inplace=True)
    path_bed = os.path.join(test_path, "test.bed")
    gff_df_bed.to_csv(path_bed, sep="\t", header=False, index=False)
    cmd = f"bedops --intersect {path_bed}"
    print(cmd)
    data = subprocess.run(cmd, shell=True, capture_output=True, text=True, universal_newlines=True, executable='/usr/bin/bash') 
    print(data.stdout)
    data_out = data.stdout
    data_out = pd.DataFrame([x.split("\t") for x in data_out.split("\n") if x])
    data_out.columns = ["qseqid", "qstart", "qend"]
    gff_creator(data_out, gff_out_path)

In [ ]:
gff_to_bed_intersect(input_df=gff_df, gff_out_path=os.path.join(test_path, "test_intersect.gff"))

In [78]:
df_4160 = gff_viewer(main_df=long_seqs, slice_num=2074, path_folder="./test_2074_v2", path_genome=path_genome)

Analyzing sequence 2074 with length 4160

    The longest sequences are:
               0        1           2     3     4  5  6  7  8  length
0   test  CBM-302  SIDER_test  2269  3546  .  .  .  .    1278
1   test  CBM-302  SIDER_test  2680  3878  .  .  .  .    1199
85  test  CBM-302  SIDER_test  2693  3828  .  .  .  .    1136
52  test  CBM-302  SIDER_test  2694  3822  .  .  .  .    1129
8   test  CBM-302  SIDER_test  2696  3823  .  .  .  .    1128
9   test  CBM-302  SIDER_test  2696  3823  .  .  .  .    1128
17  test  CBM-302  SIDER_test   734  1836  .  .  .  .    1103
16  test  CBM-302  SIDER_test   734  1836  .  .  .  .    1103
10  test  CBM-302  SIDER_test  2696  3721  .  .  .  .    1026
82  test  CBM-302  SIDER_test  2815  3828  .  .  .  .    1014
          


In [79]:
df_4160.head()


,qseqid,sseqid,sstrand,pident,qstart,qend,sstart,send,evalue,bitscore,length,qlen,qcovs,slen,mismatch,gapopen,gaps
1,Seq_LinJ.36_plus_2498408-2502567,LinJ.36,minus,99.140,2269,3546,2570590,2569313,0.000000e+00,2300.0,1279,4160,100,2743073,9,2,2
2,Seq_LinJ.36_plus_2498408-2502567,LinJ.36,minus,99.500,2680,3878,2441176,2439979,0.000000e+00,2180.0,1199,4160,100,2743073,5,1,1
3,Seq_LinJ.36_plus_2498408-2502567,LinJ.36,plus,97.535,3177,3823,2600802,2601450,0.000000e+00,1109.0,649,4160,100,2743073,14,1,2
4,Seq_LinJ.36_plus_2498408-2502567,LinJ.36,plus,97.535,3177,3823,2659295,2659943,0.000000e+00,1109.0,649,4160,100,2743073,14,1,2
5,Seq_LinJ.36_plus_2498408-2502567,LinJ.36,minus,96.944,1,360,2482923,2482573,6.250000e-169,595.0,360,4160,100,2743073,2,2,9


In [80]:
df_4160["evalue"].describe()

count    9.800000e+01
mean     2.793669e-32
std      7.559498e-32
min      0.000000e+00
25%      7.410000e-49
50%      4.805000e-39
75%      6.202500e-34
max      4.560000e-31
Name: evalue, dtype: float64